<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-03/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/diabetic_data.csv")

for c in df.columns:
  pctUnknown = np.sum([df[c] == '?'])/df[c].shape[0]
  print(c, pctUnknown)
  if(pctUnknown > 0.02):
    df.drop(c, inplace = True)
  else:
    df.drop(df[df[c] == '?'], axis = 0)

df.drop('diag_1', inplace = True)
df.drop('diag_2', inplace = True)
df.drop('diag_3', inplace = True)
df.replace('No', 0, inplace = True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
encounter_id 0.0
patient_nbr 0.0
race 0.022335554114340742
gender 0.0
age 0.0
weight 0.9685847925633315
admission_type_id 0.0
discharge_disposition_id 0.0
admission_source_id 0.0
time_in_hospital 0.0
payer_code 0.395574160328597
medical_specialty 0.49082208203132677
num_lab_procedures 0.0
num_procedures 0.0
num_medications 0.0
number_outpatient 0.0
number_emergency 0.0
number_inpatient 0.0
diag_1 0.0002063557573256294
diag_2 0.0035178743391702533
diag_3 0.013983059174970029
number_diagnoses 0.0
max_glu_serum 0.0
A1Cresult 0.0
metformin 0.0
repaglinide 0.0
nateglinide 0.0
chlorpropamide 0.0
glimepiride 0.0
acetohexamide 0.0
glipizide 0.0
glyburide 0.0
tolbutamide 0.0
pioglitazone 0.0
rosiglitazone 0.0
acarbose 0.0
miglitol 0.0
troglitazone 0.0
tolazamide 0.0
examide 0.0
citoglipton 0.0
insulin 0.0
glyburide-metformin 0.0
glipizide-metformin 0.0
glimepiride-pio

In [ ]:
class myNeuralNetwork:
  def __init__(self, num_hidden_layers, hidden_dim, train_data, train_targets, epochs, lr=0.01):
    self.epochs = epochs
    self.learning_rate = lr
    self.hidden_layers = num_hidden_layers
    self.weights = [np.random.uniform(-0.1, 0.1, (np.shape(train_data)[1], hidden_dim))]
    self.biases = [np.random.uniform(-0.1, 0.1, (hidden_dim, 1))]
    for i in range(1, num_hidden_layers):
      self.weights.append(np.random.uniform(-0.1, 0.1, (hidden_dim, hidden_dim)))
      self.biases.append(np.random.uniform(-0.1, 0.1, (hidden_dim, 1)))
    self.weights.append(np.random.uniform(-0.1, 0.1, (hidden_dim, np.shape(train_targets)[1])))
    self.biases.append(np.random.uniform(-0.1, 0.1, (np.shape(train_targets)[1])))

  def forward(self, data):
    z = []
    a = [data.T]
    for l in range(self.hidden_layers+1):
      z.append(np.matmul(self.weights[l].T, a[l]) + self.biases[l])
      if(l != self.hidden_layers):
        a.append(np.tanh(z[l]))
      else:
        a.append(np.exp(z[l])/np.sum(np.exp(z[l]), axis = 0))
    return a

  def backward(self, a):
    deltas = [0] * (self.hidden_layers+1)
    deltas[-1] = (a[-1]-self.train_targets)*(self.train_targets - a[-1])
    for l in range(self.num_hidden_layers-1, -1, -1):
      deltas[l] = np.matmul(deltas[l+1], self.weights[l])*(1-np.tanh(a[l])**2)
    return deltas


  def train(self):
    N = np.shape(self.train_data)[0]
    for e in range(self.epochs):
      a = self.forward(self, self.train_data)
      dels = self.backward(a)

      for l in range(self.hidden_layers+1):
        self.weights[l] -= self.learning_rate*(np.matmul(dels[l].T, a[l]))
        self.biases[l] -= self.learning_rate*dels[l].T

  def cost(self, preds, tars):
    return np.sum(-np.log(preds[tars]), axis=1)/np.shape(preds)[0]
